# Bayesian networks & K2 algorithm

## Introduction

A Bayesian belief-network structure $B_s$ is a directed acyclic graph in which nodes represent domain variables and arcs between nodes represent probabilistic dependencies. In order to form a complete Bayesian belief network $B$, this structure is augmented by a set of conditional probabilities, $B_f$, that relates every node to its immediate predecessors (*parents*). In the following we will use $\pi_i$ to denote the parent nodes of variable $x_i$.

We will use the term *conditional probability* to refer to a probability statement, such as $P(x_1 = 0|x_2=0)$, and the term *conditional probability assignment* to denote a numerical assignment to the probability statement.


With this structure we can explicitly represent conditional independence and dependence among events. Moreover, the joint probability of any particular instantiation of all n variables in a belief network can be calculated as:

\begin{equation}
P(X_1, ..., X_N) = \prod_{i=1}^n P(X_i|\pi_i)
\label{\eqn:1}
\tag{1}
\end{equation}


Although researchers have made substantial advances in developing the theory and application of belief networks, the actual construction of these networks often remains a difficult, time-consuming task.

In this project we present an algorithm able to find the most probable network structure, given an initial ordering of the variables. The **K2 algorithm**.


## The most probable structure


Let $D$ be a database of cases, $Z$ be the set of variables represented by $D$, and $B_{s_i}$ and $B_{s_j}$ two belief-network structures containing exactly those variables that are in $Z$. In order to rank order a set of structures by their posterior probabilities we have to compute the ratio:

\begin{equation}
\frac{P(B_{s_i}|D)}{P(B_{s_j}|D)} = \frac{P(B_{s_i},D)}{P(D)}\frac{P(D)}{P(B_{s_j},D)} = \frac{P(B_{s_i},D)}{P(B_{s_j},D)}
\label{eqn:2}
\tag{2}
\end{equation}


To compute this ratio in an efficent way, we have to make four assumptions:

1. **The database variables, which we denote as Z, are discrete.** 

    The application of this assumption yields to:

    \begin{equation}
    P(B_S,D) =\int_{B_p} P(D|B_s,B_p) f(B_p|B_s) P(B_s) dB_p
    \label{eqn:3}
    \tag{3}
    \end{equation}
    
    where $B_p$ is a vector whose values denote the conditional-probability assignments associated with belief-network structure $B_s$.


2.  **Cases occur independently, given a belief-network model.** 
    
    In this case, equation \ref{eqn:3} becomes:
    
    \begin{equation}
    P(B_S,D) =\int_{B_p} \prod_{h=1}^m \left[ P(C_h|B_s,B_p) \right] f(B_p|B_s) P(B_s) dB_p
    \label{eqn:4}
    \tag{4}
    \end{equation}
    
    where m is the number of cases in $D$ and $C_h$ is the h-th case in $D$.
    
    
3.  **There are no cases that have variables with missing values.**  


4.  **The density function $f(B_p|B_s)$ in equations \ref{eqn:3} and \ref{eqn:4} is uniform.** 


Given these assumptions, we are now ready to state an important result. 

**Theorem**

Let $Z$ be a set of n discrete variables, where a variable $x_i$ in $Z$ has $r_i$ possible value assignments: $(v^i_1,...,v^i_{r_i})$.

Let $D$ be a database of m cases, where each case contains a value assignment for each variable in $Z$.

Let $B_s$ denote a belief-network structure containing just the variables in $Z$. 

Each variable $x_i$ in $B_s$ has a set of parents, which we represent with a list of variables $\pi_i$.

Let $w_{ij}$ denote the j-th unique instantiation of $\pi_i$ relative to D.

Suppose there are $q_i$ unique instantiations of $\pi_i$. 

Define $N_{ijk}$ to be the number of cases in $D$ in which variable $x_i$ has the value $v^i_k$ and $\pi_i$ is instantiated as $w_{ij}$. 

Let $ N_{ij} = \sum_{k=1}^{r_i} N_{ijk}$

Then, given assumptions 1-4, it follows that:


\begin{equation}
P(B_S,D) = P(B_S)\prod_{i=1}^{n} \prod_{j=1}^{q_i}\frac{(r_i -1)!}{(N_{ij} + r_i -1)!}\prod_{k=1}^{r_i}N_{ijk}!
\label{eqn:5}
\tag{5}
\end{equation}


Since $P(B_S|D) \propto P(B_S,D)$, finding the $B_S$ that maximizes $P(B_S|D)$ is equivalent to finding the $B_S$ that maximizes $P(B_S,D)$. However, the number of possible structures grows exponentially with the number of nodes.


Let us assume that we can specify an ordering on all n variables, such that, if $x_i$ precedes $x_j$ in the ordering, then we do not allow structures in which there is an arc from $x_j$ to $x_i$.
In addition to a node ordering, let us assume equal priors $P(B_S) = c$.

Then, to maximize \ref{eqn:5} with this assumptions we need only to find the parent set of each variable that maximizes the second inner product.

In fact, now we have:

\begin{equation}
\underset{B_S}{\operatorname{max}} \left[P(B_S,D) \right] = c\prod_{i=1}^{n} \underset{\pi_i}{\operatorname{max}}\left[ \prod_{j=1}^{q_i}\frac{(r_i -1)!}{(N_{ij} + r_i -1)!}\prod_{k=1}^{r_i}N_{ijk}! \right]
\label{eqn:6}
\tag{6}
\end{equation}

A node $x_i$ can have at most $n-1$ nodes as parents. Thus, over all possible $B_S$ consistent with the ordering, $x_i$ can have no more than $2^{n-1}$ unique sets of parents. Therefore, the maximization on the right of \ref{eqn:6} occurs over at most $2^{n-1}$ parent sets.

In order to make the computation more feasible, we can also assume that for all distinct pairs of variables $x_i$ and $x_j$, our belief about $x_i$ having some set of parents is independent of our belief about $x_j$ having some set of parents. 


## K2 algorithm


We now describe a heuristic-search method used to maximize $P(B_S,D)$. We make the assumptions that we have an ordering on the domain variables and that, a priori, all structures are considered equally likely.

We modify the maximization operation on the right of \ref{eqn:6} to use a greedy-search method. In particular, we use an algorithm that begins by making the assumption that a node has no parents, and then adds incrementally that parent whose addition most increases the probability of the resulting structure, and when the addition of no single parent can increase the probability, we stop adding parents to the node.

We use the function:

\begin{equation}
g(i, \pi_i) = \prod_{j=1}^{q_i}\frac{(r_i -1)!}{(N_{ij} + r_i -1)!}\prod_{k=1}^{r_i}N_{ijk}!
\label{eqn:7}
\tag{7}
\end{equation}

We also use a function $Pred(x_i)$ that returns the set of nodes that precede $x_i$ in the node ordering.

We now present the pseudocode of the K2 algorithm:

<img src="K2_flow.png" width="600" height="400">


From a time complexity perspective, computing and storing the factorials requires $O(m+r-1)$ time. Since each call to **g** requires $O(mur)$ time, line 10 requires $O(mnur)$ time. Each time the **while** loop is entered it loops $O(u)$ time, and the **for** statements loops n times.

Combining these results, the overall K2 complexity is: $O(m+r-1) +O(munr)O(u)n = O(mu^2n^2r)$.

In our implementation the speed of the algorithm is improved replacing function **g** with its logarithm. Moreover, with this approach we prevent possible overflows.